In [15]:
from imp import reload

import numpy as np
import pandas as pd
import sys,math
import sistercellclass as scc; reload(scc)
import os

import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib notebook
# %matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]


In [6]:
datadir   = '/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/'
datafiles = [os.path.join(datadir,fn) for fn in os.listdir(datadir) if os.path.splitext(fn)[1][1:].upper() == 'XLS']

#datafiles = ['/home/lukas/projects/sistercells/experiments/SISTERS-NONSISTERS/SISTERS/072818POS9_ch2t_sis2.xls']

data      = scc.SisterCellData(infiles = datafiles, debugmode = False)

In [ ]:

for dataID,filename,x in data:
    #print dataID,filename
    cdivA,cdivB = data.CellDivisionTrajectory(dataID)
    #print np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])
    if dataID == 0:
        gentime = np.concatenate([np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])])
    else:
        gentime = np.concatenate([gentime,np.array(cdivA['generationtimeA']),np.array(cdivB['generationtimeB'])])
    #print

#np.savetxt('tmp.txt',gentime)
#plt.hist(gentime,range=(0,2),bins=40,log=True)
#print gentime
#print max(gentime)
#gth = dict()
#for x in np.arange(0,2.1,.05):
#    gth['{:.2f}'.format(x)] = 0
#for g in gentime:
#    gth['{:.2f}'.format(g)] += 1

#print gth


In [ ]:
def autocorrelation (x) :
    """
    Compute the autocorrelation of the signal, based on the properties of the
    power spectral density of the signal.
    """
    xp = x-np.mean(x)
    f = np.fft.fft(xp)
    p = np.array([np.real(v)**2+np.imag(v)**2 for v in f])
    pi = np.fft.ifft(p)
    return np.real(pi)[:x.size/2]/np.sum(xp**2)

In [ ]:
acf = dict()
for dataID,filename,x in data:
    a,b = data.CellDivisionTrajectory(dataID)
    for k in [kk for kk in a.keys() if kk[:4]!='time']:
        if not k[:-1] in acf.keys():
            acf[k[:-1]] = list()
        if len(a[k]) > 0:
            tmpacf = autocorrelation(a[k])
            if not np.isnan(tmpacf).any():
                acf[k[:-1]].append(tmpacf)
        if len(b[k[:-1]+'B']) > 0:
            tmpacf = autocorrelation(b[k[:-1] + 'B'])
            if not np.isnan(tmpacf).any():
                acf[k[:-1]].append(tmpacf)

acf_avg = dict()
for k in acf.keys():
    maxl  = np.max([len(a) for a in acf[k]])
    count = np.zeros(maxl)
    acf_avg[k] = np.zeros(maxl)
    for a in acf[k]:
        acf_avg[k][:len(a)] += a
        count[:len(a)] += 1.
    acf_avg[k] /= count
    print k
    #print acf_avg[k]
    plt.plot(np.arange(15),acf_avg[k][:15],label = k)
plt.plot(np.arange(15),np.zeros(15))
plt.legend()
plt.show()

plt.plot(count)

for k in acf_avg.keys():
    np.savetxt('ACF_{}.txt'.format(k),np.array([np.arange(len(acf_avg[k])),acf_avg[k]]).T)

In [ ]:
def gauss(x,mean,sigma2):
    return np.exp(-(x-mean)**2/(2.*sigma2))/np.sqrt(2*math.pi*sigma2)

maxcount = 30

countdiv = np.zeros(maxcount)
for dataID,fn,x in data:
    a,b = data.CellDivisionTrajectory(dataID)
    for t in np.array(a['generationtimeA']/.05,dtype=int):
        if t < maxcount: countdiv[t] += 1
    for t in np.array(b['generationtimeB']/.05,dtype=int):
        if t < maxcount: countdiv[t] += 1

plt.plot(np.arange(maxcount),np.log10(countdiv/np.sum(countdiv)))
plt.plot(np.arange(start=0,stop=maxcount,step=.05),np.log10(gauss(np.arange(start=0,stop=maxcount,step=.05),10.968599386592668,7.605410963650854)))
plt.ylim((-np.log10(np.sum(countdiv))-.5,1))
plt.show()

In [ ]:
m = np.sum(np.arange(maxcount) * countdiv)/np.sum(countdiv)
v = np.sum(np.arange(maxcount)**2 * countdiv)/np.sum(countdiv) - (np.sum(np.arange(maxcount)*countdiv)/np.sum(countdiv))**2
print 'mean: {}, sigma2: {}'.format(m,v)

print 'count all division events: {}'.format(np.sum(countdiv,dtype=int))


In [17]:
help(sm.tsa.ARMA)

Help on class ARMA in module statsmodels.tsa.arima_model:

class ARMA(statsmodels.tsa.base.tsa_model.TimeSeriesModel)
 |  Autoregressive Moving Average ARMA(p,q) Model
 |  
 |  Parameters
 |  ----------
 |  endog : array-like
 |      The endogenous variable.
 |  order : iterable
 |      The (p,q) order of the model for the number of AR parameters,
 |      differences, and MA parameters to use.
 |  exog : array-like, optional
 |      An optional array of exogenous variables. This should *not* include a
 |      constant or trend. You can specify this in the `fit` method.
 |  dates : array-like of datetime, optional
 |      An array-like object of datetime objects. If a pandas object is given
 |      for endog or exog, it is assumed to have a DateIndex.
 |  freq : str, optional
 |      The frequency of the time-series. A Pandas offset or 'B', 'D', 'W',
 |      'M', 'A', or 'Q'. This is optional if dates are given.
 |  
 |  
 |  Notes
 |  -----
 |  If exogenous variables are given, then th